In [2]:
import sys
sys.path.append('..')
import sql
import pandas as pd
from functools import reduce
import datetime
import matplotlib.pyplot as plt

In [13]:
q= '''with rand_advisory as (
    select distinct ecosystem, a.id as advisory_id,
                row_number() over (partition by ecosystem order by rand()) as rand_sort
    from fixing_releases fr
    join advisory a on fr.advisory_id = a.id
    join snykvuln.package p on a.package_id = p.id
    left join advisoryCVE aC on a.id = aC.advisory_id
    where year(publish_date) >= 2018
        and ecosystem != 'cocoapods'
)
    select ecosystem, advisory_id, a.package_id,
       case
            when exists(select * from advisoryCVE aC where aC.advisory_id = ra.advisory_id)
            then 'CVE'
            else 'non-CVE'
        end as if_cve
    from rand_advisory ra
    join snykvuln.advisory a on a.id = ra.advisory_id
    where rand_sort <= 50;'''
df = pd.DataFrame(sql.execute(q))
advisory = df.groupby('ecosystem')[['advisory_id']].count()
package = df.groupby('ecosystem')[['package_id']].nunique()
noncve = (df.loc[df['if_cve']=='non-CVE']).groupby('ecosystem')[['advisory_id']].count()
df=reduce(lambda x,y : pd.merge(x,y,on='ecosystem'),[advisory,package,noncve])
df.columns = ['advisory', 'package', 'non-cve']
df

,advisory,package,non-cve
ecosystem,,,
Composer,50,27,10
Go,50,43,10
Maven,50,42,7
NuGet,50,18,4
RubyGems,50,31,5
npm,50,38,14
pip,50,27,9


In [ ]:
# query used to extract random samples
''' insert into manual_sample
    with rand_cves as (
    select ecosystem, a.id as advisory_id, p.id as package_id, a.type,
           row_number() over (partition by ecosystem order by rand()) as rand_sample
    from snykvuln.advisory a
    join package p on a.package_id = p.id
    where a.id in (select advisory_id from fixing_releases)
    and year(publish_date) >= 2018
            and ecosystem != 'cocoapods'
        and a.id in (select distinct advisory_id from advisoryCVE)
    ),
    rand_noncves as (
    select ecosystem, a.id as advisory_id, p.id as package_id, a.type,
           row_number() over (partition by ecosystem order by rand()) as rand_sample
    from snykvuln.advisory a
    join package p on a.package_id = p.id
    where a.id in (select advisory_id from fixing_releases)
    and year(publish_date) >= 2018
            and ecosystem != 'cocoapods'
        and a.id not in (select distinct advisory_id from advisoryCVE)
    )
        select * from
    (select ecosystem, advisory_id, package_id, type
    from rand_cves
    where rand_sample <=25
    union
    select ecosystem, advisory_id, package_id, type
    from rand_noncves
    where rand_sample <=25) as sub; '''

In [7]:
q='''select *,
       case
            when exists(select * from advisoryCVE aC where aC.advisory_id = ms.advisory_id)
            then 'CVE'
            else 'non-CVE'
        end as if_cve
from manual_sample ms
join fixing_releases fr on ms.advisory_id = fr.advisory_id;'''
df = pd.DataFrame(sql.execute(q))
advisory = df.groupby('ecosystem')[['advisory_id']].nunique()
package = df.groupby('ecosystem')[['package_id']].nunique()
type = df.groupby('ecosystem')[['type']].nunique()
noncve = (df.loc[df['if_cve']=='non-CVE']).groupby('ecosystem')[['advisory_id']].count()
releases = df.groupby('ecosystem')[['advisory_id']].count()
df=reduce(lambda x,y : pd.merge(x,y,on='ecosystem'),[advisory,package,noncve,type,releases])
df.columns = ['advisory', 'package', 'non-cve','type','releases']
df

,advisory,package,non-cve,type,releases
ecosystem,,,,,
Composer,50,30,54,22,98
Go,50,49,29,25,67
Maven,50,49,31,28,76
NuGet,40,18,22,13,50
RubyGems,39,32,17,21,54
npm,50,44,28,20,70
pip,50,45,25,33,64
